In [1]:
library(readr)
library(dplyr)
library(ggplot2)
library(leaflet)
library(knitr)
library(formattable)
library(tidyverse)
library(stringr)
library(DT)
library(scales)
library(tidyverse)
library(reshape2)
library(grid)
library(gridExtra)
library(plotly)
library(stringi)
library(htmlwidgets)
library(maptools)
library(rgeos)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.4.2     ✔ purrr   0.2.5
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ tibble  1.4.2     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘scales’

The following object is masked from ‘package:purrr’:

    discard

The following objects are masked from ‘package:formattable’:

    comma, percent, scientific

The following object is masked from ‘package:readr’:

    col_factor


Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths


Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combi

In [2]:
oflc_data <- read.csv("/Users/genie/data/oflc/kaggle/h1b_data_fy2011_fy2018_20190309.csv", stringsAsFactors = FALSE, na.strings="", colClasses=c("emp_zip"="character","work_zip"="character","county_fips"="character","soc_code"="character","naics_code"="character"))


In [3]:
cat("data dimensions:", "\n", dim(oflc_data))

data dimensions: 
 4192087 26

In [4]:
# uscis_data1 <- read.csv("/Users/genie/data/uscis/uscis_h-1b_trends_application_volume.csv", stringsAsFactors = FALSE)
# uscis_data2 <- read.csv("/Users/genie/data/uscis/uscis_h-1b_trends_beneficiary_education.csv", stringsAsFactors = FALSE)
# uscis_data3 <- read.csv("/Users/genie/data/uscis/uscis_h-1b_trends_beneficiary_age_group.csv", stringsAsFactors = FALSE)
# uscis_data4 <- read.csv("/Users/genie/data/uscis/uscis_h-1b_trends_beneficiary_country_of_birth.csv", stringsAsFactors = FALSE)

naics_data <- read.csv("/Users/genie/data/naics_soc/naics_codes.csv", stringsAsFactors=FALSE)
soc_data <- read.csv("/Users/genie/data/naics_soc/soc_codes_v1.csv", stringsAsFactors=FALSE)

naics_data %>% filter(published_year==2017) %>% select(c("naics_code","naics_title")) -> naics_codes
naics_data %>% filter(published_year==2012 & !(naics_code %in% naics_codes$naics_code)) %>% select(c("naics_code","naics_title")) -> naics_codes_ext
naics_codes <- rbind(naics_codes, naics_codes_ext)
rm(naics_codes_ext)
naics_data %>% filter(published_year==2007 & !(naics_code %in% naics_codes$naics_code)) %>% select(c("naics_code","naics_title")) -> naics_codes_ext
naics_codes <- rbind(naics_codes, naics_codes_ext)
rm(naics_data)
rm(naics_codes_ext)

soc_data %>% filter(publication_year==2018) %>% select(c("soc_code","soc_name")) -> soc_codes
soc_data %>% filter(publication_year==2010 & !(soc_code %in% soc_codes$soc_code)) %>% select(c("soc_code","soc_name")) -> soc_codes_ext
soc_codes <- rbind(soc_codes, soc_codes_ext)
rm(soc_codes_ext)
soc_data %>% filter(publication_year==2000 & !(soc_code %in% soc_codes$soc_code)) %>% select(c("soc_code","soc_name")) -> soc_codes_ext
soc_codes <- rbind(soc_codes, soc_codes_ext)
rm(soc_data)
rm(soc_codes_ext)

In [5]:
# define some functions

calc_emp_avg_annual_wage <- function (wage_from, wage_to, wage_unit) {
    result <- ifelse((!is.na(wage_to) & wage_to>0), (wage_from+wage_to)/2 , wage_from)
    result <- ifelse(wage_unit=="Y", result, ifelse(wage_unit=="M", result*12, ifelse(wage_unit=="BW", result*26 , ifelse(wage_unit=="W", result*52, NA))))
    return(result)
}   

pct_change <- function(x) {x / lag(x) - 1}

In [6]:
# data cleaning
oflc_data$parent_emp_name <- oflc_data$emp_name

oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c("INFOSYS LIMITED","INFOSYS BPO LIMITED","INFOSYS PUBLIC SERVICES, INC.", "INFOSYS  BPO LIMITED","INFOSYS BPO LTD","INFOSYS PUBLIC SERVICES","INFOSYS BPO LTD.","INFOSYS BPO","INFOSYS TECHNOLOGIES LIMITED",
    "INFOSYS  LIMITED","INFOSYS PUBLIC SERVICES, INC","INFOSYS PUBLIC SERVICES INC","INFOSYS PUBLIC SERVICES INC.","INFOSYS, LTD.","INFOSYS CONSULTING INC.","INFOSYS WORLDWIDE, INC","INFOSYS CONSULTING, INC.")] <- "INFOSYS"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('SATYAM COMPUTER SERVICES LTD.','SATYAM COMPUTER SERVICES LTD','SATYAM COMPUTER SERVICES, LTD.','SATYAM COMPUTER SERVICES LIMITED','SATYAM, BPO LIMITED','SATYAM COMPUTER SERVICES, LTD')] <- "SATYAM COMPUTERS"
oflc_data$parent_emp_name[grepl("\\bcognizant\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "COGNIZANT"
oflc_data$parent_emp_name[grepl("\\bcapgemini\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "CAPGEMINI"
oflc_data$parent_emp_name[grepl("\\bdeloitte\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "DELOITTE"
oflc_data$parent_emp_name[grepl("\\btech mahindra\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "TECH MAHINDRA"
oflc_data$parent_emp_name[grepl("\\bwipro\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "WIPRO"
oflc_data$parent_emp_name[grepl("\\btata\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "TATA"
oflc_data$parent_emp_name[grepl("\\bibm\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "IBM"
oflc_data$parent_emp_name[grepl("\\bl&t\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE | (grepl("\\blarsen\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\btoubro\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE)] <- "L&T"
oflc_data$parent_emp_name[grepl("\\bPRICEWATERHOUSECOOPERS\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "PWC"
oflc_data$parent_emp_name[grepl("\\bernst\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\byoung\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "ERNST & YOUNG"
oflc_data$parent_emp_name[grepl("\\bqualcomm\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "QUALCOM"
oflc_data$parent_emp_name[grepl("\\bamazon\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "AMAZON"
oflc_data$parent_emp_name[grepl("\\bgoogle\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "GOOGLE"
oflc_data$parent_emp_name[grepl("\\bfacebook\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "FACEBOOK"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('APPLE INC.','APPLE INC','APPLE, INC','APPLE  INC')] <- "APPLE"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('TESLA, INC.','TESLA MOTORS, INC.','TESLA, INC','TESLA MOTORS')] <- "TESLA"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('VISA U.S.A. INC.','VISA U.S.A., INC.')] <- "VISA"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('CITI INTERNATIONAL FINANCIAL SERVICES, LLC')] <- "CITI"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('DISCOVER PRODUCTS INC.','DISCOVER GROUP INC.','DISCOVER BANK','DISCOVER GROUP INC','DISCOVER HOME LOANS INC')] <- "DISCOVER"
oflc_data$parent_emp_name[oflc_data$parent_emp_name %in% c('CHEVRON CORPORATION','CHEVRON USA INC')] <- "CHEVRON"
oflc_data$parent_emp_name[grepl("\\bmicrosoft\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "MICROSOFT"
oflc_data$parent_emp_name[grepl("\\byahoo\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "YAHOO"
oflc_data$parent_emp_name[grepl("\\boracle\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "ORACLE"
oflc_data$parent_emp_name[grepl("\\bnetflix\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "NETFLIX"
oflc_data$parent_emp_name[grepl("\\buber\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "UBER"
oflc_data$parent_emp_name[grepl("\\bairbnb\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "AIRBNB"
oflc_data$parent_emp_name[grepl("\\bcisco\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "CISCO"
oflc_data$parent_emp_name[grepl("\\bamerican express\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\blogistics\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE] <- "AMERICAN EXPRESS"
oflc_data$parent_emp_name[grepl("\\bwells fargo\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "WELLS FARGO"
oflc_data$parent_emp_name[grepl("\\bjpmorgan\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "JP MORGAN CHASE"
oflc_data$parent_emp_name[grepl("\\bbank of america\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "BANK OF AMERICA"
oflc_data$parent_emp_name[grepl("\\bcapital one\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "CAPITAL ONE"
oflc_data$parent_emp_name[grepl("\\bgoldman sach\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "GOLDMAN SACHS"
oflc_data$parent_emp_name[grepl("\\bwalmart\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "WALMART"
oflc_data$parent_emp_name[grepl("\\bexxon mobil\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "EXXON MOBIL"
oflc_data$parent_emp_name[grepl("\\bmacys\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "MACYS"
oflc_data$parent_emp_name[grepl("\\bcostco\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "COSTCO"
oflc_data$parent_emp_name[grepl("\\bhome depot\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "HOME DEPOT"
oflc_data$parent_emp_name[grepl("\\bfedex\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "FEDEX"
oflc_data$parent_emp_name[grepl("\\bverizon\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "VERIZON"
oflc_data$parent_emp_name[grepl("\\bat&t\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "AT&T"
oflc_data$parent_emp_name[grepl("\\bintel\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & (grepl("\\bpolaris\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE & grepl("\\bvital\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE & 
    grepl("\\bapg\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE & grepl("\\bcare\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE)] <- "INTEL"
oflc_data$parent_emp_name[grepl("\\bhcl\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & (grepl("\\binfoserv\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE & grepl("\\bcleantech\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==FALSE)] <- "HCL"
oflc_data$parent_emp_name[grepl("\\bigate\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "IGATE"
oflc_data$parent_emp_name[grepl("\\bfujitsu\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "FUJITSU"
oflc_data$parent_emp_name[grepl("\\baccenture\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "ACCENTURE"
oflc_data$parent_emp_name[grepl("\\bsyntel\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "SYNTEL"
oflc_data$parent_emp_name[grepl("\\bpatni\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "PATNI"
oflc_data$parent_emp_name[grepl("\\bhexaware\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\btechnologies\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "HEXAWARE TECH."
oflc_data$parent_emp_name[grepl("\\bmphasis\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "MPHASIS CORP."
oflc_data$parent_emp_name[grepl("\\bntt\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\bdata\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "NTT DATA INC."
oflc_data$parent_emp_name[grepl("\\bpeople tech\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\bgroup\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "PEOPLE TECH GROUP"
oflc_data$parent_emp_name[grepl("\\bust\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE & grepl("\\bglobal\\b", oflc_data$parent_emp_name, ignore.case=TRUE)==TRUE] <- "UST GLOBAL"


Warning message in grepl("\\bcognizant\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 704 is invalid in this locale”Warning message in grepl("\\bcognizant\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 15472 is invalid in this locale”Warning message in grepl("\\bcognizant\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 16535 is invalid in this locale”Warning message in grepl("\\bcognizant\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 17852 is invalid in this locale”Warning message in grepl("\\bcognizant\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 19511 is invalid in this locale”Warning message in grepl("\\bcapgemini\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 704 is invalid in this locale”Warning message in grepl("\\bcapgemini\\b", oflc_data$parent_emp_name, ignore.case = TRUE):
“input string 15472 is invalid in this locale”Warning message in grepl("\\bcapgemini

In [7]:
# set output directory to dump all the charts
# setwd("/Users/genie/dev/projects/github/h1b_data_analysis/out");
setwd("/Users/genie/data/oflc/odl");

In [8]:
getwd()

[1] "/Users/genie/data/oflc/odl"

In [ ]:
# Application volume analysis - DOL

oflc_data %>% group_by(fiscal_year) %>% summarise(fy_total = n()) -> data_x

oflc_data %>% filter(!is.na(case_status)) %>% group_by(fiscal_year) %>% summarise(certified=sum(case_status=="C"), denied=sum(case_status=="D"), withdrawn=sum(case_status=="W" | case_status=="CW")) -> data_y
data_y %>% gather(group, count, certified, denied, withdrawn) -> data_y

data_t <- left_join(data_y, data_x)
rm(data_x)
rm(data_y)

data_t %>% mutate(fy_share_pct=round(count/fy_total * 100, digits=2)) %>% select(c("fiscal_year","group","count","fy_share_pct"))-> data_t
data_t %>% group_by(group) %>% arrange(fiscal_year, .by_group = TRUE) %>% mutate(pct_change = round((count/lag(count) - 1)*100, digits=2) ) %>% ungroup() -> data_t

data_t$date <- paste(data_t$fiscal_year, "01", "01", sep="-")
data_t$date <- as.Date(data_t$date)

data_t$group <- factor(data_t$group, levels=c("certified", "denied", "withdrawn"), labels=c("LCA Certified", "LCA Denied", "LCA Withdrawn"))
data_t$tooltip <- paste0("<b>", data_t$group, "</b>","<br>fiscal year ~ " , data_t$fiscal_year , "<br>count ~ ", data_t$count, " (", data_t$fy_share_pct, "% of total cases)", 
ifelse(!is.na(data_t$pct_change), paste0( "<br>", data_t$pct_change, "% ", ifelse(data_t$pct_change>0, "rise", "drop")), ""))

g <- ggplot(data_t, aes(x=date, y=count/1000, col=group, group=group, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
labs(x="fiscal year", y="no of petitions (in thousands)", title="") + 
theme(axis.title.x = element_text(size=10, family="Arial"), 
      axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      axis.title.y = element_text(size=10,family="Arial"), 
      axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
      strip.background = element_blank(),
      strip.text = element_text(size=6, family="Arial"), 
      plot.title = element_text(size=10, face="bold", family="Arial"), 
      plot.subtitle = element_text(size=8, family="Arial"), 
      plot.caption = element_text(size=5, family="Arial", colour="grey"),
      legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

gg <- ggplotly(g, height=500, width=800, tooltip = c("text"))
rm(data_t)

saveWidget(gg, file = "application_volume_dol.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
           title = "Dept. of Labor (DOL) Certification Volume Trends, 2011-2018", knitrOptions = list())


In [ ]:
# Application volume analysis - USCIS

# uscis_data1 %>% mutate(fy_total=Receipts) %>% select(c("fiscal_year","fy_total")) -> data_a
# uscis_data1 %>% gather(group, count, Approvals, Receipts) -> data_b

# data_c <- left_join(data_b, data_a)
# rm(data_a)
# rm(data_b)

# data_c %>% mutate(fy_share_pct=round(count/fy_total * 100, digits=2)) %>% select(c("fiscal_year","group","count","fy_share_pct"))-> data_c
# data_c %>% group_by(group) %>% arrange(fiscal_year, .by_group = TRUE) %>% mutate(pct_change = round((count/lag(count) - 1)*100, digits=2) ) %>% ungroup() -> data_c
# data_c$fy_share_pct[data_c$fy_share_pct==100.00] <- NA

# data_c$date <- paste(data_c$fiscal_year, "01", "01", sep="-")
# data_c$date <- as.Date(data_c$date)

# data_c$group <- factor(data_c$group, levels=c("Receipts","Approvals"), labels=c("USCIS Total Receipts", "USCIS Approvals"))
# data_c$tooltip <- paste0("<b>", data_c$group, "</b>","<br>fiscal year ~ " , data_c$fiscal_year , "<br>count ~ ", data_c$count, ifelse(!is.na(data_c$fy_share_pct), paste0(" (", data_c$fy_share_pct, "% of total receipts)"), ""), 
# ifelse(!is.na(data_c$pct_change), paste0( "<br>", data_c$pct_change, "% ", ifelse(data_c$pct_change>0, "rise", "drop")), ""))

# g <- ggplot(data_c, aes(x=date, y=count/1000, col=group, group=group, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
# scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
# labs(x="fiscal year", y="no of petitions (in thousands)", title="USCIS Application Volume Trends, 2007-2017") + 
# theme(axis.title.x = element_text(size=10, family="Arial"), 
#       axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       axis.title.y = element_text(size=10,family="Arial"), 
#       axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
#       strip.background = element_blank(),
#       strip.text = element_text(size=6, family="Arial"), 
#       plot.title = element_text(size=10, face="bold", family="Arial"), 
#       plot.subtitle = element_text(size=8, family="Arial"), 
#       plot.caption = element_text(size=5, family="Arial", colour="grey"),
#       legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

# gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))
# rm(data_c)

# # saveWidget(gg, file = "application_volume_uscis.html");

# saveWidget(gg, file = "application_volume_uscis.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
#            title = "USCIS Application Volume Trends, 2007-2017", knitrOptions = list())


In [ ]:
# Application volume by education

# uscis_data1 %>% mutate(fy_total=Receipts) %>% select(c("fiscal_year","fy_total")) -> data_a

# uscis_data2 %>% 
# gather(group, count, Education_NoDiploma, Education_HighSchool, Education_CollegeLessThanOneYear, Education_CollegeMoreThanOneYear, Education_Associate, Education_Bachelors, Education_Masters, Education_ProfessionalDegree, Education_Doctorate, Education_Other) -> data_b

# data_c <- left_join(data_b, data_a)
# rm(data_a)
# rm(data_b)

# data_c %>% mutate(fy_share_pct=round(count/fy_total * 100, digits=2)) %>% select(c("fiscal_year","group","count","fy_share_pct"))-> data_c
# data_c %>% group_by(group) %>% arrange(fiscal_year, .by_group = TRUE) %>% mutate(pct_change = round((count/lag(count) - 1)*100, digits=2) ) %>% ungroup() -> data_c
# data_c$fy_share_pct[data_c$fy_share_pct==100.00] <- NA

# data_c$date <- paste(data_c$fiscal_year, "01", "01", sep="-")
# data_c$date <- as.Date(data_c$date)

# data_c$group <- factor(data_c$group, levels=c("Education_NoDiploma", "Education_HighSchool", "Education_CollegeLessThanOneYear","Education_CollegeMoreThanOneYear", "Education_Associate", "Education_Bachelors", "Education_Masters", "Education_ProfessionalDegree", "Education_Doctorate", "Education_Other"), 
#                       labels=c("No Diploma", "High School", "College <1 year", "College >1 year but no degree", "Associate","Bachelors","Masters","Professional Degree", "Doctorate", "Other"))
                              
# data_c$tooltip <- paste0("<b>", data_c$group, "</b>","<br>fiscal year ~ " , data_c$fiscal_year , "<br>count ~ ", data_c$count, ifelse(!is.na(data_c$fy_share_pct), paste0(" (", data_c$fy_share_pct, "% of total receipts)"), ""), 
# ifelse(!is.na(data_c$pct_change), paste0( "<br>", data_c$pct_change, "% ", ifelse(data_c$pct_change>0, "rise", "drop")), ""))

# g <- ggplot(data_c, aes(x=date, y=count/1000, col=group, group=group, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
# scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
# labs(x="fiscal year", y="no of petitions (in thousands)", title="Applicant Education Trends, 2007-2017") + 
# theme(axis.title.x = element_text(size=10, family="Arial"), 
#       axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       axis.title.y = element_text(size=10,family="Arial"), 
#       axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
#       strip.background = element_blank(),
#       strip.text = element_text(size=6, family="Arial"), 
#       plot.title = element_text(size=10, face="bold", family="Arial"), 
#       plot.subtitle = element_text(size=8, family="Arial"), 
#       plot.caption = element_text(size=5, family="Arial", colour="grey"),
#       legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

# gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))
# rm(data_c)

# # saveWidget(gg, file = "application_volume_education.html");
# saveWidget(gg, file = "application_volume_education.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
#            title = "Applicant Education Trends, 2007-2017", knitrOptions = list())



In [ ]:
# Application volume by age

# uscis_data1 %>% mutate(fy_total=Receipts) %>% select(c("fiscal_year","fy_total")) -> data_a

# uscis_data3 %>% 
# gather(group, count, Age_Under25, Age_25_to_34, Age_35_to_44,Age_45_to_54,Age_55_to_64,Age_Over65) -> data_b

# data_c <- left_join(data_b, data_a)
# rm(data_a)
# rm(data_b)

# data_c %>% mutate(fy_share_pct=round(count/fy_total * 100, digits=2)) %>% select(c("fiscal_year","group","count","fy_share_pct"))-> data_c
# data_c %>% group_by(group) %>% arrange(fiscal_year, .by_group = TRUE) %>% mutate(pct_change = round((count/lag(count) - 1)*100, digits=2) ) %>% ungroup() -> data_c
# data_c$fy_share_pct[data_c$fy_share_pct==100.00] <- NA

# data_c$date <- paste(data_c$fiscal_year, "01", "01", sep="-")
# data_c$date <- as.Date(data_c$date)

# data_c$group <- factor(data_c$group, levels=c("Age_Under25","Age_25_to_34","Age_35_to_44","Age_45_to_54","Age_55_to_64","Age_Over65"), 
#                       labels=c("Less than 25 years","25-34 years","35-44 years","45-54 years","55-64 years","More than 65 years"))
# data_c$tooltip <- paste0("<b>", data_c$group, "</b>","<br>fiscal year ~ " , data_c$fiscal_year , "<br>count ~ ", data_c$count, ifelse(!is.na(data_c$fy_share_pct), paste0(" (", data_c$fy_share_pct, "% of total receipts)"), ""), 
# ifelse(!is.na(data_c$pct_change), paste0( "<br>", data_c$pct_change, "% ", ifelse(data_c$pct_change>0, "rise", "drop")), ""))

# g <- ggplot(data_c, aes(x=date, y=count/1000, col=group, group=group, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
# scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
# labs(x="fiscal year", y="no of petitions (in thousands)", title="Applicant Age Trends, 2007-2017") + 
# theme(axis.title.x = element_text(size=10, family="Arial"), 
#       axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       axis.title.y = element_text(size=10,family="Arial"), 
#       axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
#       strip.background = element_blank(),
#       strip.text = element_text(size=6, family="Arial"), 
#       plot.title = element_text(size=10, face="bold", family="Arial"), 
#       plot.subtitle = element_text(size=8, family="Arial"), 
#       plot.caption = element_text(size=5, family="Arial", colour="grey"),
#       legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

# gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))
# rm(data_c)

# # saveWidget(gg, file = "application_volume_age.html");
# saveWidget(gg, file = "application_volume_age.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
#            title = "Applicant Age Trends, 2007-2017", knitrOptions = list())


In [ ]:
# # Application volume by country

# uscis_data1 %>% mutate(fy_total=Receipts) %>% select(c("fiscal_year","fy_total")) -> data_a

# uscis_data4 %>% 
# gather(group, count, IN, CN, CA, PH, KR,GB,MX,TW,JP,PK,FR,TR,DE,BR,CO,VE,NP,RU,IT,ES,Others) -> data_b

# data_c <- left_join(data_b, data_a)
# rm(data_a)
# rm(data_b)

# data_c %>% mutate(fy_share_pct=round(count/fy_total * 100, digits=2)) %>% select(c("fiscal_year","group","count","fy_share_pct"))-> data_c
# data_c %>% group_by(group) %>% arrange(fiscal_year, .by_group = TRUE) %>% mutate(pct_change = round((count/lag(count) - 1)*100, digits=2) ) %>% ungroup() -> data_c
# data_c$fy_share_pct[data_c$fy_share_pct==100.00] <- NA

# data_c$date <- paste(data_c$fiscal_year, "01", "01", sep="-")
# data_c$date <- as.Date(data_c$date)

# data_c$group <- factor(data_c$group, levels=c("IN","CN","CA","PH","KR","GB","MX","TW","JP","PK","FR","TR","DE","BR","CO","VE","NP","RU","IT","ES","Others"), 
#                       labels=c("India","China","Canada","South Korea", "Philippines","United Kingdom","Mexico","Taiwan","Japan","Pakistan",
#                       "France","Turkey","Germany","Brazil","Colombia","Venezuela","Nepal","Russia","Italy","Spain","Others"))

# data_c$tooltip <- paste0("<b>", data_c$group, "</b>","<br>fiscal year ~ " , data_c$fiscal_year , "<br>count ~ ", data_c$count, ifelse(!is.na(data_c$fy_share_pct), paste0(" (", data_c$fy_share_pct, "% of total receipts)"), ""), 
# ifelse(!is.na(data_c$pct_change), paste0( "<br>", data_c$pct_change, "% ", ifelse(data_c$pct_change>0, "rise", "drop")), ""))

# g <- ggplot(data_c, aes(x=date, y=count/1000, col=group, group=group, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
# scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
# labs(x="fiscal year", y="no of petitions (in thousands)", title="Applicant Nationality Trends, 2007-2017") + 
# theme(axis.title.x = element_text(size=10, family="Arial"), 
#       axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       axis.title.y = element_text(size=10,family="Arial"), 
#       axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
#       strip.background = element_blank(),
#       strip.text = element_text(size=6, family="Arial"), 
#       plot.title = element_text(size=10, face="bold", family="Arial"), 
#       plot.subtitle = element_text(size=8, family="Arial"), 
#       plot.caption = element_text(size=5, family="Arial", colour="grey"),
#       legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
#       legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

# gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))
# rm(data_c)

# # saveWidget(gg, file = "application_volume_nationality.html");
# saveWidget(gg, file = "application_volume_nationality.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
#            title = "Applicant Nationality Trends, 2007-2017", knitrOptions = list())


In [ ]:
library(htmltools)

oflc_data %>% filter(fiscal_year==2018 & !is.na(lat) & !is.na(lng)) %>% 
group_by(lat,lng) %>% summarise(count = length(lat)) %>% ungroup() -> data_coords

rr <- tags$div(
   HTML('<span>Nationwide job concentration, 2018</span>')
) 

gg <- leaflet(width = "100%") %>% addProviderTiles("CartoDB.Positron") %>% 
setView(-99, 35, zoom = 4) %>% 
addCircleMarkers(data=data_coords, lat = ~lat, lng = ~lng, radius=~sqrt(count)/10) %>%
addControl(rr, position = "bottomleft")

rm(data_coords)

# saveWidget(gg, file = "nationwide_h1b_job_concentration.html");
saveWidget(gg, file = "nationwide_h1b_concentration.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
           title = "Nationwide job concentration, 2018", knitrOptions = list())


In [ ]:
# # state-wide distribution

# library(maptools)
# library(fiftystater)
# library(ggiraph)

# crswgs84=CRS("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# us_states=readShapePoly("/Users/genie/data/shapefiles/us-states-shapefiles/cb_2016_us_state_500k.shp",proj4string=crswgs84,verbose=TRUE)


# #us_states <- readShapePoly(dsn = "../input/us-states-cartographic-boundary-shapefiles/cb_2016_us_state_500k.shp")

# us_states_dict <- list(
# 'AK' = 'Alaska','AL' = 'Alabama','AR' = 'Arkansas','AS' = 'American Samoa','AZ' = 'Arizona','CA' = 'California','CO' = 'Colorado','CT' = 'Connecticut','DC' = 'District of Columbia','DE' = 'Delaware','FL' = 'Florida','GA' = 'Georgia','GU' = 'Guam','HI' = 'Hawaii','IA' = 'Iowa','ID' = 'Idaho','IL' = 'Illinois','IN' = 'Indiana','KS' = 'Kansas','KY' = 'Kentucky','LA' = 'Louisiana','MA' = 'Massachusetts','MD' = 'Maryland','ME' = 'Maine','MI' = 'Michigan','MN' = 'Minnesota','MO' = 'Missouri','MP' = 'Northern Mariana Islands','MS' = 'Mississippi','MT' = 'Montana','NA' = 'National','NC' = 'North Carolina','ND' = 'North Dakota','NE' = 'Nebraska','NH' = 'New Hampshire','NJ' = 'New Jersey','NM' = 'New Mexico','NV' = 'Nevada','NY' = 'New York','OH' = 'Ohio','OK' = 'Oklahoma','OR' = 'Oregon','PA' = 'Pennsylvania','PR' = 'Puerto Rico','RI' = 'Rhode Island','SC' = 'South Carolina','SD' = 'South Dakota','TN' = 'Tennessee','TX' = 'Texas','UT' = 'Utah','VA' = 'Virginia','VI' = 'Virgin Islands','VT' = 'Vermont','WA' = 'Washington','WI' = 'Wisconsin','WV' = 'West Virginia','WY' = 'Wyoming'
# )

# data_1 <- oflc_data
# data_1$work_state_name <- tolower(us_states_dict[data_1$work_state])
# data_1 %>% filter(case_status=="C" & !is.na(work_state_name)) %>% 
# group_by(work_state_name, fiscal_year) %>% summarise(certified_count_statewide = n()) %>% 
# arrange(desc(fiscal_year)) -> data_x
# rm(data_1)

# oflc_data %>% filter(case_status=="C") %>%
# group_by(fiscal_year) %>% summarise(certified_count_nationwide = n()) %>%
# arrange(desc(fiscal_year)) -> data_y

# data_x <- left_join(data_x, data_y, by="fiscal_year")
# rm(data_y)

# gg_map <- ggplot(data_x, aes(map_id = work_state_name))
# gg_map <- gg_map + geom_map_interactive(aes(fill = certified_count_statewide,
#                   tooltip = paste(stri_trans_general(work_state_name, id="Title"), '</br>certfied cases: ', 
#                     certified_count_statewide, '</br>share ~ ', round((certified_count_statewide/certified_count_nationwide * 100), digits=2), '%'),
#                   data_id = work_state_name
#                 ),
#                 map = fifty_states) +
#     expand_limits(x = fifty_states$long, y = fifty_states$lat) +
#   coord_map() +
#   scale_x_continuous(breaks = NULL) + scale_y_continuous(breaks = NULL) +  
#   scale_fill_continuous(name="no of petitions", low = "#a6bddb", high = "#045a8d", breaks = c(25000, 50000, 75000), labels = c("25k", "50k", "75k")) +
#   labs(title="", x="", y="") + 
#   theme(legend.position = "right", 
#         legend.title=element_text(family="Arial", size="10"), 
#         legend.text=element_text(family="Arial", size="10"), 
#         strip.text=element_text(family="Arial", size="10"),
#         panel.background = element_blank())+
#   facet_wrap(~fiscal_year, ncol = 4)

# gg <- ggiraph(code = print(gg_map), width_svg=9, height_svg=4)

# rm(data_x)

# # saveWidget(gg, file = "statewide_h1b_distribution.html");
# saveWidget(gg, file = "statewide_h1b_distribution.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
#            title = "Statewide job concentration", knitrOptions = list())


In [ ]:
# derive odl statewide dist.

oflc_data %>% 
filter(!is.na(case_status)) %>%
group_by(fiscal_year) %>% 
summarise(fy_total = n(),fy_total_certified=sum(case_status=="C"), fy_total_denied=sum(case_status=="D"), 
          fy_total_withdrawn=sum(case_status=="W" | case_status=="CW")) -> data_x

data_temp <- oflc_data

data_temp %>% filter(!is.na(work_state)) %>%
group_by(fiscal_year, work_state) %>%
summarise(fy_state_total = n(),fy_state_total_certified=sum(!is.na(case_status) & case_status=="C"), 
          fy_state_total_denied=sum(!is.na(case_status) & case_status=="D"), 
          fy_state_total_withdrawn=sum(!is.na(case_status) & (case_status=="W" | case_status=="CW"))) -> data_y

data_t <- left_join(data_y, data_x)

data_t %>% mutate(fy_state_share_pct=round(fy_state_total/fy_total * 100, digits=2)) %>%
select(c("fiscal_year","work_state", "fy_state_total", "fy_state_share_pct", "fy_state_total_certified",
        "fy_state_total_denied","fy_state_total_withdrawn")) -> data_t


# Write CSV in R
write.csv(data_t, file = "oflc_state_statistics.csv",row.names=FALSE, na="")

rm(data_temp)
rm(data_x)
rm(data_y)

In [ ]:
data_x %>% filter(work_loc=="SAN FRANCISCO,CA") -> data_z
head(data_z)

In [ ]:
# popular cities

oflc_data %>% filter(fiscal_year %in% c(2011,2018) & case_status=="C" & !is.na(metro)) %>% 
group_by(fiscal_year, metro) %>% summarise(n = n()) %>% top_n(n=20) %>% ungroup() -> data_x

data_x %>% group_by(fiscal_year, metro) %>% arrange(desc(n)) %>% ungroup() %>% 
mutate(metro2 = factor(paste(metro, fiscal_year, sep = "__"), levels = rev(paste(metro, fiscal_year, sep = "__")))) -> data_x

oflc_data %>% filter(case_status=="C") %>%
group_by(fiscal_year) %>% summarise(certified_count_in_fy = n()) %>%
arrange(desc(fiscal_year)) -> data_y

data_x <- left_join(data_x, data_y, by="fiscal_year")
rm(data_y)

data_x$metro <- as.factor(data_x$metro)
data_x$n <- as.numeric(data_x$n)
data_x$fiscal_year <- as.factor(data_x$fiscal_year)

data_x$tooltip <- paste0("<b>", data_x$metro, "</b>","<br>fiscal year ~ " , data_x$fiscal_year , 
    "<br>certified count ~ ", data_x$n, "<br>share ~ ", round(data_x$n/data_x$certified_count_in_fy * 100, digits=2), "%")    

g <- ggplot(data_x, aes(metro2, n, tooltip=tooltip)) + geom_bar_interactive(stat="identity", fill="red", color="black") + 
facet_wrap(~ fiscal_year, scales = "free", ncol=2) + 
labs(x="", y="No of certifications", title="") + 
theme(axis.title.x =  element_blank(), axis.text.x  = element_text(angle=0, vjust=0.5, size=6, family="Arial"), axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=6, family="Arial"),
              strip.background = element_blank(),strip.text = element_text(size=6, family="Arial"), plot.title = element_text(size=8, face="bold", family="Arial"), 
              plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=6, family="Arial", colour="grey")) +
coord_flip() +
scale_x_discrete(labels = function(x) gsub("__.+$", "", x)) 

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=3.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)
                 
popular_metros <- levels(data_x$metro)
rm(data_x)

# saveWidget(gg, file = "popular_metros_by_certifications_volume.html");
saveWidget(gg, file = "popular_metros_by_certifications_volume.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
           knitrOptions = list())

In [ ]:
# median wage trends for selected cities

oflc_data %>% filter(case_status=="C" & !is.na(metro)) %>% 
filter(metro %in% popular_metros) %>%
# filter(!(soc_code %in% c("11-1010", "11-1011"))) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) %>%
group_by(fiscal_year, metro) %>% 
summarise(median_salary = median(emp_annual_wage, na.rm=TRUE)) %>% ungroup() -> data_x

data_x %>%
  group_by(metro) %>% 
  arrange(fiscal_year, .by_group = TRUE) %>%
  mutate(pct_change = round((median_salary/lag(median_salary) - 1)*100, digits=2) ) %>% ungroup() -> data_x

data_x$nearby_metro <- as.factor(data_x$metro)
data_x$fiscal_year1 <- paste(data_x$fiscal_year, "01", "01", sep="-")
data_x$fiscal_year1 <- as.Date(data_x$fiscal_year1)
data_x$tooltip <- paste0("<b>", data_x$metro, "</b>","<br>fiscal year ~ " , data_x$fiscal_year , "<br>median salary ~ ", currency(data_x$median_salary,digits=0L), " (", data_x$pct_change, "% change)")

g <- ggplot(data_x, aes(x=fiscal_year1, y=median_salary, col=metro, group=metro, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
labs(x="fiscal year", y="median salary", title="Median Wage Trends in Selected Metropolitan Cities, 2011-2018", 
     subtitle="Hover over dots for more info.", caption="Source: Dept. of Labor ~ H-1B Disclosure Data") + 
theme(axis.title.x = element_text(size=10, family="Arial"), 
      axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      axis.title.y = element_text(size=10,family="Arial"), 
      axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
      strip.background = element_blank(),
      strip.text = element_text(size=6, family="Arial"), 
      plot.title = element_text(size=10, face="bold", family="Arial"), 
      plot.subtitle = element_text(size=8, family="Arial"), 
      plot.caption = element_text(size=5, family="Arial", colour="grey"),
      legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))

rm(data_x)

# saveWidget(gg, file = "selected_metros_median_wage_analysis.html");
saveWidget(gg, file = "selected_metros_median_wage_analysis.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
           knitrOptions = list())


In [ ]:
# wage segments for selected cities

oflc_data %>% filter(fiscal_year==2018 & case_status=="C" & !is.na(metro)) %>% 
filter(metro %in% popular_metros) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) %>% 
filter(!is.na(emp_annual_wage)) -> data_x

data_x$emp_wage_bin <- cut(data_x$emp_annual_wage, breaks = c(0, 60000, 85000, 100000, 125000, 150000, 200000, Inf), right=FALSE, labels = c("<60k","60-85k","85-100k","100-125k","125-150k","150-200k",">200k"))
data_x$emp_wage_bin <- factor(data_x$emp_wage_bin)

data_x %>% group_by(metro) %>% summarise(c_count_by_city = n()) -> data_y
data_x %>% group_by(metro, emp_wage_bin) %>% summarise(c_count_in_group = n()) -> data_z

data_t <- left_join(data_z, data_y)
rm(data_y)
rm(data_z)
rm(data_x)

data_t %>% mutate(pct_of_total = round(c_count_in_group/c_count_by_city*100, digits=2)) -> data_t

data_t$tooltip <- paste0("<b>", data_t$metro, "</b>","<br>wage segment ~ " , data_t$emp_wage_bin , "<br>certified count ~ ", data_t$c_count_in_group, " (", data_t$pct_of_total, "%)")

g <- ggplot(data_t, aes(x=emp_wage_bin, y=c_count_in_group, fill=emp_wage_bin, tooltip=tooltip)) + 
geom_bar_interactive(stat="identity") +
facet_wrap(~metro, scales = "free", ncol=4) + 
labs(x="", y="", title="Salary Segmentation in Selected Cities, 2018") + 
theme(axis.title.x=element_blank(), axis.text.x=element_blank(),
    #axis.text.x=element_text(angle=90, vjust=0.5, size=5, family="Arial"), 
    axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=5, family="Arial"),
    strip.background = element_blank(), strip.text = element_text(size=5, family="Arial"), 
    plot.title = element_text(size=8, face="bold", family="Arial"), plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=5, family="Arial", colour="grey"), 
    legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.4, "cm"), legend.text=element_text(size=4, family="Arial"), legend.title=element_text(size=6, family="Arial")) +
guides(fill=guide_legend(title="Wage Segments"))

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=8.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)

rm(data_t)

# saveWidget(gg, file = "selected_metros_wage_segment_analysis.html");
saveWidget(gg, file = "selected_metros_wage_segment_analysis.html", selfcontained = FALSE, libdir = "chart_resources", background = "white", 
           knitrOptions = list())


In [ ]:
# popular employers

oflc_data %>% filter(case_status=="C" & !is.na(parent_emp_name)) %>% 
group_by(fiscal_year, parent_emp_name) %>% summarise(n = n()) %>% top_n(n=20) %>% ungroup() -> data_x

data_x$parent_emp_name <- as.factor(data_x$parent_emp_name)
data_x$n <- as.numeric(data_x$n)
data_x$fiscal_year <- as.factor(data_x$fiscal_year)

data_x %>% group_by(fiscal_year, parent_emp_name) %>% arrange(desc(n)) %>% ungroup() %>% 
mutate(parent_emp_name2 = factor(paste(parent_emp_name, fiscal_year, sep = "__"), levels = rev(paste(parent_emp_name, fiscal_year, sep = "__")))) -> data_x

data_x$tooltip <- paste0("<b>", data_x$parent_emp_name, "</b>","<br>total certified cases in fiscal year " , data_x$fiscal_year , " ~ ", data_x$n)    

g <- ggplot(data_x, aes(parent_emp_name2, n, tooltip=tooltip)) + geom_bar_interactive(stat="identity", fill="red", color="black") + 
facet_wrap(~ fiscal_year, scales = "free", ncol=1) + 
labs(x="", y="", title="Top-20 Popular Employers for H-1B workers, 2011-2017") + 
theme(axis.title.x =  element_blank(), axis.text.x  = element_text(angle=0, vjust=0.5, size=6, family="Arial"), axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=6, family="Arial"),
              strip.background = element_blank(),strip.text = element_text(size=6, family="Arial"), plot.title = element_text(size=8, face="bold", family="Arial"), 
              plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=6, family="Arial", colour="grey")) +
coord_flip() +
scale_x_discrete(labels = function(x) gsub("__.+$", "", x)) 

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.75,height=15.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)
                 
rm(data_x)
                 
saveWidget(gg, file="popular_employers_by_certifications_volume.html")           

In [ ]:
# median wage trends for selected employers

selected_employers <- c("INFOSYS","WIPRO","COGNIZANT","TATA","DELOITTE","CAPGEMINI","ACCENTURE","IBM","QUALCOMM","GOOGLE","FACEBOOK","AMAZON","MICROSOFT","YAHOO",
"APPLE","INTEL","ORACLE", "TESLA", "AT&T","VERIZON","CISCO","NETFLIX","UBER","AIRBNB","AMERICAN EXPRESS","BANK OF AMERICA","WELLS FARGO","JP MORGAN CHASE","CAPITAL ONE")

oflc_data %>% filter(case_status=="C" & !is.na(parent_emp_name)) %>% 
filter(parent_emp_name %in% selected_employers) %>%
filter(!(soc_code %in% c("11-1010", "11-1011"))) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) %>%
group_by(fiscal_year, parent_emp_name) %>% 
#summarise(median_salary = median(emp_annual_wage, na.rm=TRUE)) %>% ungroup() -> data_x
summarise(n = n(), median_salary=median(emp_annual_wage, na.rm=TRUE), per_capita=sum(emp_annual_wage, na.rm=TRUE)/n()) %>% ungroup() -> data_x

data_x$parent_emp_name <- as.factor(data_x$parent_emp_name)
data_x$fiscal_year1 <- paste(data_x$fiscal_year, "01", "01", sep="-")
data_x$fiscal_year1 <- as.Date(data_x$fiscal_year1)
data_x$tooltip <- paste0("<b>", data_x$parent_emp_name, "</b>","<br>fiscal year ~ " , data_x$fiscal_year , "<br>certified count ~", data_x$n, "<br>median salary ~ ", currency(data_x$median_salary,digits=0L), "<br>per capita ~ ", currency(data_x$per_capita,digits=0L))

g <- ggplot(data_x, aes(x=fiscal_year1, y=median_salary, col=parent_emp_name, group=parent_emp_name, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
labs(x="fiscal year", y="median salary", title="Median Salary Trends for Selected Employers, 2011-2017") + 
theme(axis.title.x = element_text(size=10, family="Arial"), 
      axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      axis.title.y = element_text(size=10,family="Arial"), 
      axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
      strip.background = element_blank(),
      strip.text = element_text(size=6, family="Arial"), 
      plot.title = element_text(size=10, face="bold", family="Arial"), 
      plot.subtitle = element_text(size=8, family="Arial"), 
      plot.caption = element_text(size=5, family="Arial", colour="grey"),
      legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))

rm(data_x)

saveWidget(gg, file = "selected_employers_median_wage_analysis.html");

In [ ]:
# wage segments for selected employers

oflc_data %>% filter(fiscal_year==2017 & case_status=="C" & !is.na(parent_emp_name)) %>% 
filter(parent_emp_name %in% selected_employers) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) -> data_x

data_x$emp_wage_bin <- cut(data_x$emp_annual_wage, breaks = c(0, 60000, 85000, 100000, 125000, 150000, 200000, Inf), right=FALSE, labels = c("<60k","60-85k","85-100k","100-125k","125-150k","150-200k",">200k"))
data_x$emp_wage_bin <- factor(data_x$emp_wage_bin)

data_x %>% group_by(parent_emp_name) %>% summarise(c_count_by_employer = n()) -> data_y
data_x %>% group_by(parent_emp_name, emp_wage_bin) %>% summarise(c_count_in_group = n()) -> data_z

data_t <- left_join(data_z, data_y)
rm(data_y)
rm(data_z)
rm(data_x)

data_t %>% mutate(pct_of_total = round(c_count_in_group/c_count_by_employer*100, digits=2)) -> data_t

data_t$tooltip <- paste0("<b>", data_t$parent_emp_name, "</b>","<br>wage segment ~ " , data_t$emp_wage_bin , "<br>certified count ~ ", data_t$c_count_in_group, " (", data_t$pct_of_total, "%)")

g <- ggplot(data_t, aes(x=emp_wage_bin, y=c_count_in_group, fill=emp_wage_bin, tooltip=tooltip)) + 
geom_bar_interactive(stat="identity") +
facet_wrap(~parent_emp_name, scales = "free", ncol=4) + 
labs(x="", y="", title="Salary Segmentation in Selected Employers, 2017") + 
theme(axis.title.x=element_blank(), axis.text.x=element_blank(),
    #axis.text.x=element_text(angle=90, vjust=0.5, size=5, family="Arial"), 
    axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=5, family="Arial"),
    strip.background = element_blank(), strip.text = element_text(size=5, family="Arial"), 
    plot.title = element_text(size=8, face="bold", family="Arial"), plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=5, family="Arial", colour="grey"), 
    legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.4, "cm"), legend.text=element_text(size=4, family="Arial"), legend.title=element_text(size=6, family="Arial")) +
guides(fill=guide_legend(title="Wage Segments"))

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=8.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)

rm(data_t)

saveWidget(gg, file = "selected_employers_wage_segment_analysis.html");



In [ ]:
# popular industries

oflc_data %>% filter(case_status=="C" & !is.na(naics_code)) %>% 
group_by(fiscal_year, naics_code) %>% summarise(n = n()) %>% top_n(n=20) %>% ungroup() -> data_x

data_x$naics_code <- as.factor(data_x$naics_code)
data_x$n <- as.numeric(data_x$n)
data_x$fiscal_year <- as.factor(data_x$fiscal_year)

data_x %>% group_by(fiscal_year, naics_code) %>% arrange(desc(n)) %>% ungroup() %>% 
mutate(naics_code2 = factor(paste(naics_code, fiscal_year, sep = "__"), levels = rev(paste(naics_code, fiscal_year, sep = "__")))) -> data_x

data_x <- merge(data_x, naics_codes, by="naics_code")
data_x$tooltip <- paste0("<b>", data_x$naics_code, ":", data_x$naics_title, "</b>","<br>certified cases", " ~ ", data_x$n)    

popular_industries <- levels(data_x$naics_code)

g <- ggplot(data_x, aes(naics_code2, n, tooltip=tooltip)) + geom_bar_interactive(stat="identity", fill="red", color="black") + 
facet_wrap(~ fiscal_year, scales = "free", ncol=2) + 
labs(x="", y="", title="Top-20 Popular Industries for H-1B workers, 2011-2017") + 
theme(axis.title.x =  element_blank(), axis.text.x  = element_text(angle=0, vjust=0.5, size=6, family="Arial"), axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=6, family="Arial"),
              strip.background = element_blank(),strip.text = element_text(size=6, family="Arial"), plot.title = element_text(size=8, face="bold", family="Arial"), 
              plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=6, family="Arial", colour="grey")) +
coord_flip() +
scale_x_discrete(labels = function(x) gsub("__.+$", "", x)) 

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=8.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)
                 
rm(data_x)
                 
saveWidget(gg, file = "popular_industries_by_certifications_volume.html");

In [ ]:
# median wage trends for selected industries

oflc_data %>% filter(case_status=="C" & !is.na(naics_code)) %>% 
filter(naics_code %in% popular_industries) %>%
filter(!(soc_code %in% c("11-1010", "11-1011"))) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) %>%
group_by(fiscal_year, naics_code) %>% 
summarise(median_salary = median(emp_annual_wage, na.rm=TRUE)) %>% ungroup() -> data_x

data_x$naics_code <- as.factor(data_x$naics_code)
data_x$fiscal_year1 <- paste(data_x$fiscal_year, "01", "01", sep="-")
data_x$fiscal_year1 <- as.Date(data_x$fiscal_year1)

data_x <- merge(data_x, naics_codes, by="naics_code")
data_x$tooltip <- paste0("<b>", data_x$naics_code, "-",  data_x$naics_title ,"</b>","<br>fiscal year ~ " , data_x$fiscal_year , "<br>median salary ~ ", currency(data_x$median_salary,digits=0L))

g <- ggplot(data_x, aes(x=fiscal_year1, y=median_salary, col=naics_code, group=naics_code, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
labs(x="fiscal year", y="median salary", title="Median Salary Trends in Selected Industries, 2011-2017") + 
theme(axis.title.x = element_text(size=10, family="Arial"), 
      axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      axis.title.y = element_text(size=10,family="Arial"), 
      axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
      strip.background = element_blank(),
      strip.text = element_text(size=6, family="Arial"), 
      plot.title = element_text(size=10, face="bold", family="Arial"), 
      plot.subtitle = element_text(size=8, family="Arial"), 
      plot.caption = element_text(size=5, family="Arial", colour="grey"),
      legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))

rm(data_x)

saveWidget(gg, file = "selected_industries_median_wage_analysis.html");


In [ ]:
# wage segments for selected industries

oflc_data %>% filter(fiscal_year==2017 & case_status=="C" & !is.na(naics_code)) %>% 
filter(naics_code %in% popular_industries) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) -> data_x

data_x$emp_wage_bin <- cut(data_x$emp_annual_wage, breaks = c(0, 60000, 85000, 100000, 125000, 150000, 200000, Inf), right=FALSE, labels = c("<60k","60-85k","85-100k","100-125k","125-150k","150-200k",">200k"))
data_x$emp_wage_bin <- factor(data_x$emp_wage_bin)

data_x %>% group_by(naics_code) %>% summarise(c_count_by_industry = n()) -> data_y
data_x %>% group_by(naics_code, emp_wage_bin) %>% summarise(c_count_in_group = n()) -> data_z

data_t <- left_join(data_z, data_y)
rm(data_y)
rm(data_z)
rm(data_x)

data_t %>% mutate(pct_of_total = round(c_count_in_group/c_count_by_industry*100, digits=2)) -> data_t

data_t <- merge(data_t, naics_codes, by="naics_code")
data_t$tooltip <- paste0("<b>", data_t$naics_code, "-",  data_t$naics_title ,"</b>","<br>wage segment ~ " , data_t$emp_wage_bin , "<br>certified count ~ ", data_t$c_count_in_group, " (", data_t$pct_of_total, "%)")

g <- ggplot(data_t, aes(x=emp_wage_bin, y=c_count_in_group, fill=emp_wage_bin, tooltip=tooltip)) + 
geom_bar_interactive(stat="identity") +
facet_wrap(~naics_code, scales = "free", ncol=4) + 
labs(x="", y="", title="Salary Segmentation in Selected Industries, 2017") + 
theme(axis.title.x=element_blank(), axis.text.x=element_blank(),
    #axis.text.x=element_text(angle=90, vjust=0.5, size=5, family="Arial"), 
    axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=5, family="Arial"),
    strip.background = element_blank(), strip.text = element_text(size=5, family="Arial"), 
    plot.title = element_text(size=8, face="bold", family="Arial"), plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=5, family="Arial", colour="grey"), 
    legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.4, "cm"), legend.text=element_text(size=4, family="Arial"), legend.title=element_text(size=6, family="Arial")) +
guides(fill=guide_legend(title="Wage Segments"))

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=8.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)

rm(data_t)

saveWidget(gg, file = "selected_industries_wage_segment_analysis.html");


In [ ]:
# popular occupations

oflc_data %>% filter(case_status=="C" & !is.na(soc_code)) %>% 
group_by(fiscal_year, soc_code) %>% summarise(n = n()) %>% top_n(n=20) %>% ungroup() -> data_x

data_x$soc_code <- as.factor(data_x$soc_code)
data_x$n <- as.numeric(data_x$n)
data_x$fiscal_year <- as.factor(data_x$fiscal_year)

data_x %>% group_by(fiscal_year, soc_code) %>% arrange(desc(n)) %>% ungroup() %>% 
mutate(soc_code2 = factor(paste(soc_code, fiscal_year, sep = "__"), levels = rev(paste(soc_code, fiscal_year, sep = "__")))) -> data_x

data_x <- merge(data_x, soc_codes, by="soc_code")
data_x$tooltip <- paste0("<b>", data_x$soc_code, ":",  data_x$soc_name ,"</b>","<br>certified cases count", " ~ ", data_x$n)    

popular_occupations <- levels(data_x$soc_code)

g <- ggplot(data_x, aes(soc_code2, n, tooltip=tooltip)) + geom_bar_interactive(stat="identity", fill="red", color="black") + 
facet_wrap(~ fiscal_year, scales = "free", ncol=2) + 
labs(x="", y="", title="Top-20 Popular Occupations for H-1B workers, 2011-2017") + 
theme(axis.title.x =  element_blank(), axis.text.x  = element_text(angle=0, vjust=0.5, size=6, family="Arial"), axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=6, family="Arial"),
              strip.background = element_blank(),strip.text = element_text(size=6, family="Arial"), plot.title = element_text(size=8, face="bold", family="Arial"), 
              plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=6, family="Arial", colour="grey")) +
coord_flip() +
scale_x_discrete(labels = function(x) gsub("__.+$", "", x)) 

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=8.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)
                 
rm(data_x)

saveWidget(gg, file = "popular_occ_by_certifications_volume.html");

In [ ]:
# median wage trends for selected occupations

oflc_data %>% filter(case_status=="C" & !is.na(soc_code)) %>% 
filter(soc_code %in% popular_occupations) %>%
filter(!(soc_code %in% c("11-1010", "11-1011"))) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) %>%
group_by(fiscal_year, soc_code) %>% 
summarise(median_salary = median(emp_annual_wage, na.rm=TRUE)) %>% ungroup() -> data_x

data_x$soc_code <- as.factor(data_x$soc_code)
data_x$fiscal_year1 <- paste(data_x$fiscal_year, "01", "01", sep="-")
data_x$fiscal_year1 <- as.Date(data_x$fiscal_year1)

data_x <- merge(data_x, soc_codes, by="soc_code")
data_x$tooltip <- paste0("<b>", data_x$soc_code, "-",  data_x$soc_name ,"</b>","<br>fiscal year ~ " , data_x$fiscal_year , "<br>median salary ~ ", currency(data_x$median_salary,digits=0L))

g <- ggplot(data_x, aes(x=fiscal_year1, y=median_salary, col=soc_code, group=soc_code, text=tooltip)) + geom_line(size=0.4) + geom_point(size=0.6) +
scale_x_date(date_breaks = "1 year", date_labels="%Y") + 
labs(x="fiscal year", y="median salary", title="Median Salary Trends in Selected Occupations, 2011-2017") + 
theme(axis.title.x = element_text(size=10, family="Arial"), 
      axis.text.x = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      axis.title.y = element_text(size=10,family="Arial"), 
      axis.text.y  = element_text(angle=0, vjust=0.5, size=8, family="Arial"),
      strip.background = element_blank(),
      strip.text = element_text(size=6, family="Arial"), 
      plot.title = element_text(size=10, face="bold", family="Arial"), 
      plot.subtitle = element_text(size=8, family="Arial"), 
      plot.caption = element_text(size=5, family="Arial", colour="grey"),
      legend.title = element_blank(), legend.text = element_text(angle=0, vjust=0.5, size=8, family="Arial"), 
      legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.3, "cm"))

gg <- ggplotly(g, height=550, width=900, tooltip = c("text"))

rm(data_x)

saveWidget(gg, file = "selected_occ_median_wage_analysis.html");

In [ ]:
# wage segments for selected occupations

oflc_data %>% filter(fiscal_year==2017 & case_status=="C" & !is.na(soc_code)) %>% 
filter(soc_code %in% popular_occupations) %>%
mutate(emp_annual_wage = calc_emp_avg_annual_wage(wage_from, wage_to, wage_unit)) -> data_x

data_x$emp_wage_bin <- cut(data_x$emp_annual_wage, breaks = c(0, 60000, 85000, 100000, 125000, 150000, 200000, Inf), right=FALSE, labels = c("<60k","60-85k","85-100k","100-125k","125-150k","150-200k",">200k"))
data_x$emp_wage_bin <- factor(data_x$emp_wage_bin)

data_x %>% group_by(soc_code) %>% summarise(c_count_by_occupation = n()) -> data_y
data_x %>% group_by(soc_code, emp_wage_bin) %>% summarise(c_count_in_group = n()) -> data_z

data_t <- left_join(data_z, data_y)
rm(data_y)
rm(data_z)
rm(data_x)

data_t %>% mutate(pct_of_total = round(c_count_in_group/c_count_by_occupation*100, digits=2)) -> data_t

data_t <- merge(data_t, soc_codes, by="soc_code")
data_t$tooltip <- paste0("<b>", data_t$soc_code, "-",  data_t$soc_name ,"</b>","<br>wage segment ~ " , data_t$emp_wage_bin , "<br>certified count ~ ", data_t$c_count_in_group, " (", data_t$pct_of_total, "%)")

g <- ggplot(data_t, aes(x=emp_wage_bin, y=c_count_in_group, fill=emp_wage_bin, tooltip=tooltip)) + 
geom_bar_interactive(stat="identity") +
facet_wrap(~soc_code, scales = "free", ncol=4) + 
labs(x="", y="", title="Salary Segmentation in Selected Occupations, 2017") + 
theme(axis.title.x=element_blank(), axis.text.x=element_blank(),
    #axis.text.x=element_text(angle=90, vjust=0.5, size=5, family="Arial"), 
    axis.title.y = element_text(face="bold", size=12), axis.text.y  = element_text(angle=0, vjust=0.5, size=5, family="Arial"),
    strip.background = element_blank(), strip.text = element_text(size=5, family="Arial"), 
    plot.title = element_text(size=8, face="bold", family="Arial"), plot.subtitle = element_text(size=6, family="Arial"), plot.caption = element_text(size=5, family="Arial", colour="grey"), 
    legend.position="bottom", legend.spacing=unit(0.3, "cm"), legend.key.size = unit(0.4, "cm"), legend.text=element_text(size=4, family="Arial"), legend.title=element_text(size=6, family="Arial")) +
guides(fill=guide_legend(title="Wage Segments"))

tooltip_css <- "background-color:black;color:white;font-family:Arial;font-size:15px;padding:5px;"
gg <- ggiraph(code = {print(g)}, width=0.9,height=8.0, tooltip_offx = 20, tooltip_offy = -10, tooltip_extra_css = tooltip_css)

rm(data_t)

saveWidget(gg, file = "selected_occ_wage_segment_analysis.html");